In [1]:
%load_ext autoreload
%autoreload 2

from tqdm import tqdm
import pandas as pd
import numpy as np
import gzip
import json
import math
from datetime import datetime
from IPython.display import display
%matplotlib inline

In [ ]:
from helper.manage_orderbooks_v2 import extract_orderbooks_for_one_currencypair, load_orderbook_snapshot
from helper.orderbook_container import OrderbookContainer
from helper.orderbook_trader import OrderbookTradingSimulator

In [ ]:
filename = '3000orderbooks'
data = load_orderbook_snapshot(filename, last_line=100)

 47%|████▋     | 47/100 [00:02<00:02, 21.15it/s]

In [ ]:
orderbook = data[0]
print(orderbook)
orderbook2 = data[1]
print(orderbook2)

print("\nCompare two orderbooks_")
diff = orderbook.compare_with(other=orderbook2)

display(diff.head())
print(diff)

In [ ]:
vol = 40
print("Current MarketOrderPrice for {} shares: {}".format(vol, orderbook.get_current_price(vol)))

In [ ]:
display(orderbook.head(2))
display(orderbook.tail(2))

In [ ]:
ots = OrderbookTradingSimulator()
vol = 40
T=10
P = 2
for e in range(T):
    print("epoch {}".format(e))
    res = ots.trade_timespan(orderbooks=data[P*e:P*(e+1)], volume=vol, limit=707.5, timespan=P, verbose=True)
    vol = ots.history.volume_left.values[-1]
# res = ots.trade_timespan(orderbooks=data[2:4], volume=40, limit=705.5, timespan=2)

display(ots.history)
display(ots.buy_history)
display(res.head(5))